In [3]:
#截取2017年的
import pandas as pd
data = pd.read_csv('./US_Accidents_Dec20_Updated.csv')
datacopy = data.copy()
datacopy['Start_Time'] = pd.to_datetime(datacopy['Start_Time'])
datacopy['year'] = datacopy['Start_Time'].apply(lambda x:x.year)
data1 = datacopy[datacopy['year']==2017]
data1.to_csv('./USaccident2017.csv')

In [6]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import folium
import pandas as pd
import  webbrowser
from pyecharts import options as opts
from pyecharts.charts import Page, Pie, Bar, Line, Scatter
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [7]:
data = pd.read_csv('./USaccident2017.csv')
data.shape #(717483, 51)
data.head()

,Unnamed: 0,ID,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,year
0,13,A-14,2,2017-08-16 17:26:10,2017-08-16 17:55:57,32.265141,-110.903587,32.265141,-110.903587,0.00,...,False,False,False,False,False,Day,Day,Day,Day,2017
1,26,A-27,3,2017-09-07 20:37:46,2017-09-07 21:07:21,32.871090,-80.010628,32.871090,-80.010628,0.00,...,False,False,False,False,False,Night,Night,Night,Day,2017
2,33,A-34,2,2017-10-20 09:23:35,2017-10-20 10:53:09,30.420996,-91.140549,30.420996,-91.140549,0.00,...,False,False,False,True,False,Day,Day,Day,Day,2017
3,67,A-68,3,2017-04-18 10:21:04,2017-04-18 10:51:04,32.857914,-96.651512,NaN,NaN,0.01,...,False,False,False,False,False,Day,Day,Day,Day,2017
4,68,A-69,3,2017-08-14 08:33:36,2017-08-14 09:17:53,30.334675,-81.670998,30.334675,-81.670998,0.00,...,False,False,False,False,False,Day,Day,Day,Day,2017


In [10]:
#处理空值
#无影响或者不分析的列 删除
deletelist= ['Unnamed: 0', 'ID', 'End_Lat', 'End_Lng', 'Airport_Code','Weather_Timestamp','Wind_Chill(F)',
             'Civil_Twilight', 'Nautical_Twilight',
             'Astronomical_Twilight', 'year','Number'#,'TMC'
            ]
data1 = data.drop(deletelist, axis=1)
#删除有空值的行
data1 = data1.dropna(axis = 0,subset=['City','Zipcode','Timezone','Sunrise_Sunset'])
#温度湿度气压能见度用均值填补
data1['Temperature(F)'] = data1['Temperature(F)'].fillna(data1['Temperature(F)'].mean())
data1['Humidity(%)'] = data1['Humidity(%)'].fillna(data1['Humidity(%)'].mean())
data1['Pressure(in)'] = data1['Pressure(in)'].fillna(data1['Pressure(in)'].mean())
data1['Visibility(mi)'] = data1['Visibility(mi)'].fillna(data1['Visibility(mi)'].mean())
#风速使用近邻填补
data1['Wind_Speed(mph)'] = data1['Wind_Speed(mph)'].interpolate(method='nearest', order=4)
#天气状况风向用众数填补
data1['Weather_Condition'] = data1['Weather_Condition'].fillna(data1['Weather_Condition'].mode())
data1['Wind_Direction'] = data1['Wind_Direction'].fillna(data1['Wind_Direction'].mode())
#降水量没有就用0填补
data1['Precipitation(in)'] = data1['Precipitation(in)'].fillna(0)
#风向，把同样含义单词的合并起来
occupation = {"CALM":"Calm", "N":"North", "S":"South", "W":"West", "E":"East", "VAR":"Variable"}
f = lambda x : occupation.get(x,x) #在occupation中找对应的值
data1['Wind_Direction']= data1['Wind_Direction'].map(f)
#最后矫正索引因为删除了部分行
data1.index = range(len(data1))

In [11]:
#发生事故最多的州
a=(
    Bar(init_opts=opts.InitOpts(width="2000px",height="400px"))
.add_xaxis(data1['State'].value_counts().index.tolist())
.add_yaxis('各州事故发生数量',data1['State'].value_counts().tolist(),color='#499C9F')
.set_series_opts(label_opts=opts.LabelOpts(is_show= False))
)
a.render_notebook()


In [12]:
#事故发生时间
x1=pd.DatetimeIndex(data1["Start_Time"]).hour.value_counts().sort_index().index.tolist()
x=[str(i) for i in x1] #pyehchart需要字符类型
from pyecharts.charts import Line
b = (Line(init_opts=opts.InitOpts(width="1000px",height="400px"))
     .add_xaxis(x)
    .add_yaxis('各时间事故发生数',pd.DatetimeIndex(data1["Start_Time"]).hour.value_counts().sort_index().tolist() ,color='#F7BA0B',is_smooth=True)
    .set_series_opts(label_opts=opts.LabelOpts(is_show= False), markarea_opts=opts.MarkAreaOpts(
            data=[opts.MarkAreaItem(name="早高峰", x=("6", "9")),opts.MarkAreaItem(name="晚高峰", x=("15", "18"))]))
     .set_global_opts(xaxis_opts=opts.AxisOpts(name='时间/时',name_location = "center",name_gap= 40))
    )
b.render_notebook()

In [13]:
#事故发生月份
x1=pd.DatetimeIndex(data1["Start_Time"]).month.value_counts().sort_index().index.tolist()
x=[str(i) for i in x1] #pyehchart需要字符类型
from pyecharts.charts import Line
q = (Line(init_opts=opts.InitOpts(width="1000px",height="400px"))
     .add_xaxis(x)
    .add_yaxis('月份事故发生数',pd.DatetimeIndex(data1["Start_Time"]).month.value_counts().sort_index().tolist() ,
               color='#AED54C',
               is_smooth=True,
               areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
              )
     .set_global_opts(xaxis_opts=opts.AxisOpts(name='month',name_location = "center",name_gap= 40))
    )
q.render_notebook()


In [14]:
#各天气下发生事故数量
weather10 = data1['Weather_Condition'].value_counts().head(10)
c=(
    Bar()
.add_xaxis(weather10.index.tolist())
.add_yaxis('各天气下发生事故数量',weather10.tolist(),color='#48A43F')
.set_series_opts(label_opts=opts.LabelOpts(is_show= False))
.set_global_opts(
xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15))#解决标签名太长
))
c.render_notebook()

In [15]:
#天气晴朗下发生事故类型
Clear_wearther = data1[:][data1['Weather_Condition']=='Clear']
occupation = {1:"轻微事故", 2:"一般事故", 3:"较大事故", 4:"重大事故"}
f = lambda x : occupation.get(x,x) #在occupation中找对应的值
Clear_wearther['Severity']= Clear_wearther['Severity'].map(f)
Clear_wearther['Severity'].value_counts().index
d = (Pie()
        .add("hotel",[list(z) for z in zip(['一般事故', '较大事故', '重大事故', '轻微事故'],Clear_wearther['Severity'].value_counts().tolist())])
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
)
d.render_notebook()

In [16]:
#查看发生事故时各环境条件
#是否是白天 1就是白天
m = lambda x : 1 if x=='Day' else 0
data1['Sunrise_Sunset'] = data1['Sunrise_Sunset'].apply(m)
#是否有降雨
m = lambda x : 1 if x>0 else 0
data1['PrecipitationORnot'] = data1['Precipitation(in)'].apply(m)
df0=pd.concat([data1['Crossing'].value_counts()
               ,data1['PrecipitationORnot'].value_counts()
               ,data1['Sunrise_Sunset'].value_counts().sort_index()
               ,data1['Traffic_Signal'].value_counts()
               ,data1['Give_Way'].value_counts()
               ,data1['Bump'].value_counts()
              ],axis=1)

h = (
    Bar()
    .add_xaxis(['是否路口','有无降水','是否白天','有无信号灯','有无让路标志','有无减速带'])
    .add_yaxis("0", df0.loc[False].tolist(), stack="stack1",color='#992572')
    .add_yaxis("1",df0.loc[True].tolist(), stack="stack1",color='#4A203B')
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title="Bar-堆叠数据（全部）"))

)
h.render_notebook()

In [17]:
#事故发生时能见度
#按百度百科的能见度表分级
data1["Visibility_bin"] = "差"
data1.loc[(data1["Visibility(mi)"]>2.5)&(data1["Visibility(mi)"]<=6.5), "Visibility_bin"] = "中等"
data1.loc[(data1["Visibility(mi)"]>6.5)&(data1["Visibility(mi)"]<=12), "Visibility_bin"] = "良好"
data1.loc[(data1["Visibility(mi)"]>12), "Visibility_bin"] = "很好"
d = (Pie()
        .add("hotel",[list(z) for z in zip(['良好', '中等', '差', '很好'],data1["Visibility_bin"].value_counts().tolist())])
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
         .set_global_opts(title_opts=opts.TitleOpts(title="事故发生时能见度"))

)
d.render_notebook()

In [18]:
# 这里使用folium库画美国地图并在data1中随机取3000个事故地点在地图上打点
incidents = folium.map.FeatureGroup()
datasample=data1.sample(3000) 
# Loop through the 200 crimes and add each to the incidents feature group
for lat, lng, in zip(datasample.Start_Lat,datasample.Start_Lng):
    incidents.add_child(
        folium.CircleMarker(
            [lat, lng],
            radius=3, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='red',
            fill_opacity=0.4
        )
    )

# Add incidents to map
US_map = folium.Map(location=[38, -100], zoom_start=4)
US_map.add_child(incidents)

In [19]:
dataX = data1.copy()
#提取月份和小时
dataX['month'] = dataX['Start_Time'].apply(lambda x:x.month)
dataX['hour'] = dataX['Start_Time'].apply(lambda x:x.hour)
删除对建模无用的特征
deletelist2=['Source','Side','Start_Time', 'End_Time','Description',
             'Street','City','County', 'State', 'Zipcode', 'Country', 'Timezone','Wind_Direction']
dataX = dataX.drop(deletelist2, axis=1)
#把false换成0 true换成1
list3=['Amenity',
       'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway',
       'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal','Turning_Loop']
m = lambda x : 1 if x else 0
for i in list3:
    dataX[i] = dataX[i].apply(m)
#严重程度为1的数量太少极度不均衡，删掉
dataX = dataX.drop(index=(dataX.loc[(dataX['Severity']==1)].index))
dataX.index = range(len(dataX))
dataX.head()

AttributeError: 'str' object has no attribute 'month'

In [20]:
y = dataX['Severity']#标签列
Xw=dataX['Weather_Condition']#需要独热编码的列
X = dataX.drop(['Severity','Weather_Condition'],axis=1)
#独热编码
enc = OneHotEncoder(categories='auto',handle_unknown='ignore').fit(Xw.values.reshape(-1,1))
result = enc.transform(Xw.values.reshape(-1,1)).toarray()
Xw1=pd.DataFrame(result)
Xw1.shape #(71669,78)

ValueError: Input contains NaN